# Big Data Platform
## Assignment 3: ServerLess

**By:**  

John Doe, 300123123  
Jane Doe, 200123123

<br><br>

**The goal of this assignment is to:**
- Understand and practice the details of Serverless

**Instructions:**
- Students will form teams of two people each, and submit a single homework for each team.
- The same score for the homework will be given to each member of your team.
- Your solution is in the form of a Jupyter notebook file (with extension ipynb).
- Images/Graphs/Tables should be submitted inside the notebook.
- The notebook should be runnable and properly documented. 
- Please answer all the questions and include all your code.
- You are expected to submit a clear and pythonic code.
- You can change functions signatures/definitions.

**Submission:**
- Submission of the homework will be done via Moodle by uploading (not Zip):
    - Jupyter Notebook
    - 2 Log files
    - Additional local scripts
- The homework needs to be entirely in English.
- The deadline for submission is on Moodle.
- Late submission won't be allowed.

  
- In case of identical code submissions - both groups will get a Zero. 
- Some groups might be selected randomly to present their code.

**Requirements:**  
- Python 3.6 should be used.  
- You should implement the algorithms by yourself using only basic Python libraries (such as numpy,pandas,etc.)

<br><br><br><br>

**Grading:**
- Q0 - 10 points - Setup
- Q1 - 40 points - Serverless MapReduceEngine
- Q2 - 20 points - MapReduce job to calculate inverted index
- Q3 - 30 points - Shuffle

`Total: 100`

<br><br>

# Question 0
## Setup

1. Navigate to IBM Cloud and open a trial account. No need to provide a credit card
2. Choose IBM Cloud Object Storage service from the catalog
3. Create a new bucket in IBM Cloud Object Storage
4. Create credentials for the bucket with HMAC (access key and secret key)
5. Choose IBM Cloud Functions service from the catalog and create a service


#### Lithops setup
1. By using “git” tool, install master branch of the Lithops project from
https://github.com/lithops-cloud/lithops
2. Follow Lithops documentation and configure Lithops against IBM Cloud Functions and IBM Cloud Object Storage
3. Configure Lithops log level to be in DEBUG mode
4. Run Hello World example by using Futures API and verify all is working properly.


#### IBM Cloud Object Storage setup
1. Upload all the input CSV files that you used in homework 2 into the bucket you created in IBM Cloud Object Storage


<br><br><br>

In [1]:
!pip install git+https://github.com/lithops-cloud/lithops.git

  Cloning https://github.com/lithops-cloud/lithops.git to /private/var/folders/lg/pbb4ypp52kx1shc2q76nnnlc0000gn/T/pip-req-build-rm8ibuza


  Created wheel for lithops: filename=lithops-2.5.9.dev0-py3-none-any.whl size=313590 sha256=d96265e9efcb28e84435f2a70d1831d82667feacc2e6ae5e63620180b529cc53
  Stored in directory: /private/var/folders/lg/pbb4ypp52kx1shc2q76nnnlc0000gn/T/pip-ephem-wheel-cache-byx_yd6t/wheels/f2/7a/47/12c1c1acee8bccb6b8e4b06ee26d13fe6f304831832bb2a33d
Successfully built lithops


In [2]:
!lithops test

2022-01-04 21:31:41,616 [INFO] lithops.config -- Lithops v2.5.9.dev0
2022-01-04 21:31:41,618 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-01-04 21:31:41,618 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-01-04 21:31:41,618 [INFO] lithops.invokers -- ExecutorID 98ffac-0 | JobID A000 - Selected Runtime: python 
2022-01-04 21:31:41,620 [INFO] lithops.invokers -- ExecutorID 98ffac-0 | JobID A000 - Starting function invocation: hello() - Total: 1 activations
2022-01-04 21:31:41,930 [INFO] lithops.invokers -- ExecutorID 98ffac-0 | JobID A000 - View execution logs at /private/var/folders/lg/pbb4ypp52kx1shc2q76nnnlc0000gn/T/lithops/logs/98ffac-0-A000.log
2022-01-04 21:31:41,931 [INFO] lithops.wait -- ExecutorID 98ffac-0 - Getting results from functions

  100%|██████████████████████████████████████████████████████████████████| 1/1  

2022-01-04 21:31:43,964 [INFO] lithops.executors -- ExecutorID 98ffac-0 - Cleaning te

In [17]:
!pip install lithops aws

     |████████████████████████████████| 53 kB 690 kB/s eta 0:00:01
     |████████████████████████████████| 212 kB 2.0 MB/s eta 0:00:01


  Created wheel for aws: filename=aws-0.2.5-py3-none-any.whl size=7461 sha256=b5baff69bc74dfc513f83fd0da1c4bd7cd7145f5e6cb78f8480487213f703f54
  Stored in directory: /Users/ah11500/Library/Caches/pip/wheels/bc/92/fb/ee4b27c55c4e55e43d3fbb1f89699aeda162d3bf700fbba01a
Successfully built aws


In [24]:
!pip install boto3

     |████████████████████████████████| 131 kB 636 kB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 571 kB/s eta 0:00:01
     |████████████████████████████████| 79 kB 4.5 MB/s eta 0:00:01


In [25]:
import lithops
import os

In [26]:
AWSAccessKeyId = os.environ.get("AWSAccessKeyId")
AWSSecretKey = os.environ.get("AWSSecretKey")

In [40]:
config = {'lithops': {'backend': 'aws_lambda', 'storage': 'aws_s3', 'log_level':'DEBUG'},

          'aws':  {'access_key_id': AWSAccessKeyId,
                   'secret_access_key': AWSSecretKey},

          'aws_s3': {'storage_bucket': 'idc-big-data',
                     'region_name': 'eu-central-1'},
          
          'aws_lambda': {'execution_role': 'arn:aws:iam::981299761374:role/lithops-execution-role',
                     'region_name': 'eu-central-1'}}

In [41]:
def hello(name):
    return 'Hello {}!'.format(name)

with lithops.FunctionExecutor(config=config) as fexec:
    fut = fexec.call_async(hello, 'World')
    print(fut.result())

2022-01-04 22:43:12,259 [INFO] lithops.config -- Lithops v2.5.9.dev0
2022-01-04 22:43:12,264 [INFO] lithops.storage.backends.aws_s3.aws_s3 -- S3 client created - Region: eu-central-1
2022-01-04 22:43:13,070 [INFO] lithops.serverless.backends.aws_lambda.aws_lambda -- AWS Lambda client created - Region: eu-central-1
2022-01-04 22:43:13,071 [INFO] lithops.invokers -- ExecutorID 9f2ec3-11 | JobID A000 - Selected Runtime: python38 - 256MB
2022-01-04 22:43:13,869 [INFO] lithops.invokers -- ExecutorID 9f2ec3-11 | JobID A000 - Starting function invocation: hello() - Total: 1 activations
2022-01-04 22:43:13,872 [INFO] lithops.invokers -- ExecutorID 9f2ec3-11 | JobID A000 - View execution logs at /private/var/folders/lg/pbb4ypp52kx1shc2q76nnnlc0000gn/T/lithops/logs/9f2ec3-11-A000.log
2022-01-04 22:43:13,879 [INFO] lithops.storage.backends.aws_s3.aws_s3 -- S3 client created - Region: eu-central-1


Hello World!


# Question 1
## Serverless MapReduceEngine

Modify MapReduceEngine from homework 2 into the MapReduceServerlessEngine where map and reduce tasks executed as a serverless actions, instead of local threads. In particular:
1. Deploy all map tasks as a serverless actions by using Lithops against IBM Cloud Functions.
2. Collect results from all map tasks and store them in the same SQLite as you used in MapReduceEngine and use the same code for the sort and shuffle phase.
3. Deploy reduce tasks by using Lithops against IBM Cloud Functions. Instead of persisting results from reduce tasks, return results back to the MapReduceServerlessEngine and proceed with the same workflow as in MapReduceEngine
4. Return results of reduce tasks to the user

**Please attach:**  
Text file with all log messages Lithops printed to console during the execution. Make
sure log level is set to DEBUG mode.

#### Code:

In [35]:
def create_connection(db_data):
    try:
        conn = sqlite3.connect(db_data)
        print("Establish connection")
    except Error as e:
        print(e)
    finally:
        return conn


def create_table(conn, create_table_query):
    try:
        c = conn.cursor()
        c.execute(create_table_query)
    except Error as e:
        print(e)
        
def query(conn, select_query):
    try:
        c = conn.cursor()
        select_all = select_query
        rows = c.execute(select_all).fetchall()
        # Output to the console screen
        return rows
    
    except Error as e:
        print(e)

In [51]:
# import boto3 
# from boto.s3.connection import S3Connection
# s3 = S3Connection(AWSAccessKeyId, AWSSecretKey)
# bucket = s3.get_bucket('s3://idc-big-data')
# print(bucket)

In [52]:
MYDATA_DB = 'file:s3://idc-big-data/db/temp_results.db'

create_table_temp_results = '''CREATE TABLE IF NOT EXISTS temp_results(
                                key TEXT,
                                value TEXT)
                            '''

In [53]:
fexec.call_async(create_connection, MYDATA_DB)

# conn = create_connection(MYDATA_DB)
# create_table(conn,create_table_temp_results)

2022-01-04 23:01:33,968 [INFO] lithops.invokers -- ExecutorID 9f2ec3-11 | JobID A003 - Selected Runtime: python38 - 256MB
2022-01-04 23:01:33,969 [INFO] lithops.invokers -- ExecutorID 9f2ec3-11 | JobID A003 - Starting function invocation: create_connection() - Total: 1 activations
2022-01-04 23:01:33,970 [INFO] lithops.invokers -- ExecutorID 9f2ec3-11 | JobID A003 - View execution logs at /private/var/folders/lg/pbb4ypp52kx1shc2q76nnnlc0000gn/T/lithops/logs/9f2ec3-11-A003.log


In [ ]:
import concurrent.futures as cf

class MapReduceEngine():
    
    def execute(self, input_data, map_function, reduce_function, params):
                        
        # map function
        threads = []
        
        for count, csv_file in enumerate(input_data):
            with cf.ThreadPoolExecutor() as executor:
                t = executor.submit(map_function, csv_file, params)             
                df = t.result()
                df.to_csv(f'mapreducetemp/part-tmp-{count+1}.csv', index=False)
                threads.append(t)
                
        # Check if all threads are completed:   
        if len(threads) != len(input_data):
            print("MapReduce Failed")
            return

            
        threads.clear()

        # Load contect of all CSV files into the temp_results table
        for i in range(len(input_data)):
            data = pd.read_csv(f'mapreducetemp/part-tmp-{i+1}.csv')
            data.to_sql('temp_results',conn, if_exists='append',index=False)
        
        # SQL statements:
        grouping_query = "SELECT key, GROUP_CONCAT(value,',') FROM temp_results GROUP BY key"
        reduce_input_values = query(conn,grouping_query)
        
        unique_keys = "SELECT  count(distinct key) FROM temp_results"
        reduce_threads = query(conn,unique_keys)[0][0]
        
        # reduce function
        threads = []
        
        for i in range(reduce_threads):
            with cf.ThreadPoolExecutor() as executor:
                key = reduce_input_values[i][0]
                values = reduce_input_values[i][1].split(',')
                
                t = executor.submit(reduce_function, key, values)             
                df = t.result()
                df.to_csv(f'mapreducefinal/part-{i+1}-final.csv', index=False)
                threads.append(t)
        
        # Check if all threads are completed:   
        if len(threads) != reduce_threads:
            print("MapReduce Failed")
            return
        
        else:
            print("MapReduce Completed")         

In [ ]:
# The goal of map function is to create csv files. 
# Therfore we chose to optimize the process and not create unnecessary variables (such as list) and return a data frame. 

def inverted_map(document_name:str, column_index:dict):
    csv_dir = f'csv_files/{document_name}'
    file = pd.read_csv(csv_dir)
    df = pd.DataFrame({'key':file.iloc[:,column_index['column']],'value':csv_dir})
    return df 


In [ ]:
# The goal of reduce function is to create csv files. 
# Therfore we chose to optimize the process and not create unnecessary variables (such as list) and return a data frame. 

def inverted_reduce(key,documents):
    df = pd.DataFrame({'key':key,'value':list(set(documents))})
    return df



In [ ]:
from os import listdir

def find_csv_filenames(path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [ ]:
input_data = find_csv_filenames('csv_files/')

In [ ]:
mapreduce = MapReduceEngine()
params = {'column':2}
status = mapreduce.execute(input_data, inverted_map, inverted_reduce, params)

# Task 2
## Submit MapReduce job to calculate inverted index
1. Use input_data: `cos://bucket/<path to CSV data>`
2. Submit MapReduce job with reduce and map functions as you used in homework 2, as follows

    `mapreduce = MapReduceServerlessEngine()`  
    `results = mapreduce.execute(input_data, inverted_map, inverted_index)`   
    `print(results)`

**Please attach:**  
Text file with all log messages Lithops printed to console during the execution. Make
sure log level is set to DEBUG mode.

#### Code:

# Question 3
## Shuffle

MapReduceServerlessEngine deploys both map and reduce tasks as serverless invocations.   
However, once map stage completed, the result are transferred from the map tasks to the SQLite database located on the client machine (laptop in your case), then performed local shuffle and then invoked reduce tasks passing them relevant parameters.

(To support your answers, feel free to use examples, Images, etc.)
<br><br>

**1. Explain why this approach is not efficient and what are cons and pros of such architecture in general. In broader scope you may assume that MapReduceServerlessEngine executed in some powerful machine and not just laptop.**

\<your answer here>

<br><br>
**2. Suggest how can you improve shuffle so intermediate data will not be downloaded to the client at all and shuffle performed in the cloud as well. Explain pros and cons of the approaches you suggest.**


\<your answer here>

<br><br>
**3. Can you make serverless shuffle?**


\<your answer here>

<br><br><br><br>
Good Luck :) 